# Langchain #3: (예제) 웹사이트 문서요약

In [38]:
import os
os.environ['OPENAI_API_KEY'] = '안알려줌'

In [5]:
from dotenv import load_dotenv
load_dotenv()

False

# 웹문서 요약

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

url = "https://entertain.naver.com/ranking/read?oid=477&aid=0000472375"
loader = WebBaseLoader(url)
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000, chunk_overlap=300, length_function=len, is_separator_regex=False
)

docs = WebBaseLoader(url).load_and_split(text_splitter)
docs

[Document(page_content='[단독]최동석 맹비난한 \'술파티\', 유기동물 자선 행사였다…박지윤 경매 진행 :: 네이버 TV연예\n\n\n메인 메뉴로 바로가기\n본문으로 바로가기\n\n\n검색\n\n검색\n\n\n검색창 닫기\n\n\nNAVER\nTV연예\n\n뉴스\n\n\nTV연예홈\nTV\n포토\n랭킹\n영화\n최신뉴스\n\n\n\t\t\t[단독]최동석 맹비난한 \'술파티\', 유기동물 자선 행사였다…박지윤 경매 진행\n\t\t\n\n기사입력2024.02.06. 오전 10:46\n최종수정2024.02.06. 오전 11:14\n기사원문\n\n\n좋아요\n\n좋아요 평가하기\n공감\n\n\n좋아요\n0\n\n\n응원해요\n0\n\n\n축하해요\n0\n\n\n기대해요\n0\n\n\n놀랐어요\n0\n\n\n슬퍼요\n0\n\n\n글꼴\n\n나눔고딕\n맑은고딕\n돋움체\n바탕체\n\n\n본문 사이즈 작게\n본문 사이즈 크게\n\n인쇄하기\n\n \n▲ 박지윤(왼쪽) 최동석. 출처|티빙, KBS[스포티비뉴스=장진리 기자] 방송인 최동석이 파경을 맞은 전처 박지윤에게 "아픈 아이들을 놔두고 술파티에 갔다"라고 맹비난을 쏟아낸 가운데, 박지윤이 \'술자리\'가 아닌 유기동물 구조를 위한 자선 행사 진행을 맡았던 것으로 전해졌다. 6일 스포티비뉴스 취재에 따르면 박지윤은 지난 4일 유기동물 구조, 치료, 입양을 지원하는 민트의 자선 행사에 참석했다.이 자선 행사는 참여자들이 낸 식사, 와인 비용과 경매 행사 수익금을 안성에 있는 평강공주 보호소에 기부하는 이벤트였다. 평강공주 보호소는 각지에서 구조된 강아지 220여 마리와 고양이 150여 마리를 돌보는 곳이다. 배우 공승연이 스페셜 호스트를 맡았고, 박지윤은 방송인의 역량을 살려 경매 행사 진행을 맡았다. 이 행사에 대해 최동석은 박지윤이 아픈 아이들을 두고 사적인 술자리를 가졌다고 힐난해 파문을 불러일으켰다. 최동석은 5일 "밤 늦은 시간이 되니 SNS 여기저기서 올라오는 파티 사진들, 와인잔을 기울이며 

# 템플릿 정의

* LLM이 허용하는 최대 토큰의 수보다 많은 경우, 쪼개서 여러 번 물어보는 `map_reduce` 방식으로 물어보자.

In [31]:
# 쪼개진 chunk 단위
template = '''한국어로 요약해줘:

{text}'''

# 전체 chunk에 대한 지시 정의
combine_template = '''{text}

요약 결과를 아래와 같이 작성해줘:
제목: 신문기사의 제목
주요내용: 한 줄로 요약된 내용
내용: 주요 내용을 불렛포인트 형식으로 작성
평가: 이 신문기사에 대한 너의 의견
'''

prompt = PromptTemplate(template=template, input_variables=['text'])
combine_prompt = PromptTemplate(template=combine_template, input_variables=['text'])

# LLM 및 체인 정의

`chain_type` 매개변수
* `stuff`: 모든 프롬프트 텍스트를 단일호출로 LLM에 보낸다.
* `map_reduce`: 여러 청크로 분할하여 LLM에 보낸 후, 마지막에 결합한다.

In [33]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0.8, callbacks=[StreamingStdOutCallbackHandler()])

# 장문의 웹사이트 본문 내용을 요약
chain = load_summarize_chain(
    llm, map_prompt=prompt, combine_prompt=combine_prompt, chain_type='map_reduce', verbose=False
)

print(chain.run(docs))

제목: 최동석-박지윤 갈등, 술파티 논란
주요내용: 최동석이 박지윤을 비난한 '술파티'는 유기동물 자선 행사였고, 박지윤은 경매 진행을 맡았다.
내용:
- 최동석이 박지윤을 비난한 '술파티'는 실제로 유기동물 자선 행사였다.
- 박지윤은 경매 진행을 맡아 유기동물을 돕는 활동을 하고 있었다.
- 자선 행사는 기부와 경매 수익금을 평강공주 보호소에 기부했다.
평가: 이 기사는 사람들이 선입견 없이 사건을 이해할 수 있도록 잘 정리된 것 같다. 유기동물에 대한 관심을 높일 수 있는 긍정적인 소식이다.
